# Experimentation and troubleshooting for Dan, part 2

Any important learnings should get documented/folded into the tutorial somehow!

In [1]:
import ray
import os

# os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION']='python'
ray.shutdown()

if ray.is_initialized() == False:
    service_host = os.environ["RAY_HEAD_SERVICE_HOST"]
    service_port = os.environ["RAY_HEAD_SERVICE_PORT"]
    ray.init(f"ray://{service_host}:{service_port}")

In [2]:
import numpy as np
import pandas as pd
import modin.pandas as mpd
import pyarrow.parquet as pq
import pyarrow as pa

In [3]:
default_dataset_path = f"/domino/datasets/local/{os.environ['DOMINO_PROJECT_NAME']}"

### Super small data

In [5]:
current_file = os.path.join(default_dataset_path, "smallest.parquet")
print(f"Size on disk: {os.path.getsize(current_file)/(1024*1024*1024)}")

Size on disk: 0.0009336844086647034


#### Try with ray data

In [6]:
ds = ray.data.read_parquet(current_file)

In [7]:
ds.show()

{'0': -0.8455730423087089, '1': -0.23197710872317917}
{'0': 1.57761716047386, '1': 0.2675177338719423}
{'0': -0.8469077067951065, '1': 0.465946115922862}
{'0': -0.2599499527630411, '1': 0.3014887616609474}
{'0': 1.3234030346884524, '1': 0.15941609326756717}
{'0': -1.0240875522442838, '1': 0.39425745117734895}
{'0': 0.648801815688615, '1': -0.7965117535042872}
{'0': 2.2855822508088357, '1': -1.1802913831079955}
{'0': 0.18913283448364113, '1': 0.7695369116084659}
{'0': -0.0788594455077964, '1': 1.323074897792383}
{'0': -0.5236116065318258, '1': 0.4145725741026637}
{'0': 1.5998534579655097, '1': 0.5672429283015951}
{'0': -0.147272367770004, '1': 1.4975889063298724}
{'0': 0.0814997379332405, '1': 0.23757410722604605}
{'0': -1.5257650039225852, '1': 0.43788273234547326}
{'0': 0.3568537733911725, '1': -1.8123722222147145}
{'0': 1.5319747191972874, '1': 0.7408738042753721}
{'0': 0.709873304889898, '1': 2.1425536615555045}
{'0': 0.2089996265404489, '1': 0.7608666464907928}
{'0': -0.51966259233

### Goldilocks data

In [4]:
current_file = os.path.join(default_dataset_path, "goldilocks.parquet")
print(f"Size on disk: {os.path.getsize(current_file)/(1024*1024*1024)}")

Size on disk: 0.7503557354211807


In [5]:
#ds = ray.data.read_parquet(current_file)
ds = ray.data.read_parquet(current_file, parallelism = 2)

In [6]:
ds.show()

{'0': 0.8486725261757402, '1': -1.6249991954356522, '2': 0.3491570059240676, '3': -0.3845927848310576, '4': -1.7535548830533385, '5': -1.7415539866200016, '6': -0.06332342022296654, '7': -1.2783767682189873, '8': -1.6925961944833545, '9': 1.3495421016506435, '10': 0.5923182238453245, '11': 0.4341769539541526, '12': 2.2231331215606147, '13': 1.1358900966802883, '14': -0.14083763849360784, '15': -1.7700207958491847, '16': 1.4353060846880865, '17': -0.8130187065371839, '18': -0.8799310305137245, '19': -2.0199181029533215}
{'0': 1.3195784578309429, '1': -0.37837809775452397, '2': -0.2789803290710002, '3': -1.1319832239231604, '4': -0.532566660761417, '5': -0.530425986153597, '6': 0.64005241852152, '7': -0.09528005746394203, '8': 1.3393781704897154, '9': 0.5457471334720579, '10': 0.6154668627082043, '11': 0.07025999845420669, '12': 3.373880186735437, '13': 0.36443392326697027, '14': -0.36758903818152144, '15': 1.1488883352672523, '16': 0.37827831033563736, '17': -1.016511292241403, '18': 1.

In [7]:
ds2 = ds.repartition(2)

Repartition: 100%|██████████| 2/2 [00:06<00:00,  3.39s/it]


In [8]:
del ds

In [9]:
ds3 = ray.data.read_parquet(current_file, parallelism = 2)

In [11]:
ds3.show()

2022-08-03 17:17:36,726	ERROR dataclient.py:150 -- Unrecoverable error in data channel.


ConnectionError: GRPC connection failed: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.NOT_FOUND
	details = "Failed to serialize response!"
	debug_error_string = "{"created":"@1659547057.228317745","description":"Error received from peer ipv4:172.20.241.144:10001","file":"src/core/lib/surface/call.cc","file_line":1074,"grpc_message":"Failed to serialize response!","grpc_status":5}"
>